In [ ]:
import warnings

import rasterio
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm
import matplotlib
from figures import pltconfig # varibles arent imported, this just sets plt.rcParams
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import *
from shapely.errors import ShapelyDeprecationWarning

from scrollstats import RASTER_PATHS, BendDataset, calculate_ridge_metrics

In [ ]:
plt.rcParams["font.family"]

## Get Data

In [ ]:
bend_id = "LBR_025"
bend_ds = BendDataset(bend_id)
transects = bend_ds.get_transects(proj=True)
ridges = bend_ds.get_ridges(proj=True)
packets= bend_ds.get_packets(proj=True)

dem = rasterio.open(RASTER_PATHS[bend_id]["dem"])
bin_raster = rasterio.open(RASTER_PATHS[bend_id]["bin"])


In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", ShapelyDeprecationWarning)
    rich_transects, itx = calculate_ridge_metrics(transects, bin_raster, dem, ridges)

itx = itx.loc[bend_id]

In [ ]:
itx.head()

# Plot with Histograms

In [ ]:
def agg_by_x(itx, col, num_bins=20, agg_func="mean"):
    """Aggregate the values at each itx to a given number of columns."""

    # Get bounds of the itx dataframe
    minx, miny, maxx, maxy = itx.total_bounds

    # Get x coords of the bins
    bin_xcoords = np.linspace(minx, maxx, num_bins)

    # Create array of bounding boxes for x axis
    geoms = [box(x1, miny, x2, maxy) for x1, x2 in zip(bin_xcoords[:-1], bin_xcoords[1:])]

    xbins = gpd.GeoDataFrame({"geometry":geoms}, crs = itx.crs)
    agg_values = xbins.sjoin(itx)[col].groupby(level=0).agg(agg_func)

    # Join agg values back onto geometries
    xbin_agg = xbins.join(agg_values)
    
    return xbin_agg

In [ ]:
def agg_by_y(itx, col, num_bins=20, agg_func="mean"):
    """Aggregate the values at each itx to a given number of columns."""

    # Get bounds of the itx dataframe
    minx, miny, maxx, maxy = itx.total_bounds

    # Get y coords of the bins
    bin_ycoords = np.linspace(miny, maxy, num_bins)

    # Create array of bounding boxes for y axis
    geoms = [box(minx, y1, maxx, y2) for y1, y2 in zip(bin_ycoords[:-1], bin_ycoords[1:])]

    ybins = gpd.GeoDataFrame({"geometry":geoms}, crs = itx.crs)
    agg_values = ybins.sjoin(itx)[col].groupby(level=0).agg(agg_func)

    # Join agg values back onto geometries
    ybin_agg = ybins.join(agg_values)
    
    return ybin_agg

In [ ]:
# fig = plt.figure(figsize=(5,4))
# gs = GridSpec(4,5, figure=fig)

# # Main Itx Plot
# vmin = 0
# vmax = itx["ridge_amp"].max()

# ## Create Colorbar
# ax1 = fig.add_subplot(gs[1:, :4])
# divider = make_axes_locatable(ax1)
# cax = divider.append_axes("left", size="5%", pad=0.1, aspect=2)

# ## Plot itx and ridges
# itx.plot(ax=ax1, column="ridge_amp", markersize="ridge_width", vmin=vmin, vmax=vmax, legend=True, cax=cax)
# ridges.plot(ax=ax1, color="k", ls="--", lw=0.5, zorder=0)

# min_max = [vmin, vmax]
# cax.set_yticks(min_max, labels = np.round(min_max, 1))
# cax.set_ylabel("Amplitude [m]")

# cax.yaxis.set_ticks_position('left')
# cax.yaxis.set_label_position('left')

# ax1.set_axis_off()

# # Top hist
# ax2 = fig.add_subplot(gs[0, :4])
# xbins = agg_by_x(itx, "ridge_amp", 20, "mean")
# xbins["ridge_amp"].T.plot(kind="bar", ax=ax2, width=0.8, color="mediumseagreen")
# ax2.set_xticks([])
# ax2.set_yticklabels(["0m", "2m"])
# ax2.set_xlim(-3.5, 21.5)
# ax2.spines["top"].set_color(None)
# ax2.spines["right"].set_color(None)


# # Right Hist
# ax3 = fig.add_subplot(gs[1:, 4])
# ybins = agg_by_y(itx, "ridge_amp", 20, "mean")
# ybins["ridge_amp"].plot(kind="barh", ax=ax3, width=0.8, color="mediumseagreen")
# ax3.set_yticks([])
# ax3.set_xticklabels(["0m", "2m"])
# ax3.set_ylim(-4.5, 24.8)
# ax3.spines["top"].set_color(None)
# ax3.spines["right"].set_color(None)

# # Top box
# ax4 = fig.add_subplot(gs[0, -1])
# ax4.text(0.5, 0.5, "Mean\nAmplitude", horizontalalignment="center", verticalalignment="center")
# ax4.set_xticks([])
# ax4.set_yticks([])
# ax4.set_axis_off()

# # Add x and x bins as a grid
# xbins.boundary.plot(ax=ax1, color='grey', lw=0.2, alpha=0.5, zorder=0)
# ybins.boundary.plot(ax=ax1, color='grey', lw=0.2, alpha=0.5, zorder=0)

# # plt.tight_layout()
# plt.subplots_adjust(wspace=0.1, hspace=0.1)
# # plt.savefig("/Users/avan/FLUD/BrazosScrolls/figures/RidgeAmplitudeJointPlot_colorbar.png", dpi=300)


## Create box plots of ridge metrics

In [ ]:
ridge_metrics = itx[["ridge_amp", "ridge_width", "pre_mig_dist"]]
ridge_metrics.columns = ridge_metrics.columns.rename("metrics")
ridge_metrics

In [ ]:
ridge_metrics_per_transect = ridge_metrics.unstack(level=0)

amps_per_transect = ridge_metrics_per_transect["ridge_amp"]
widths_per_transect = ridge_metrics_per_transect["ridge_width"]
spacing_per_transect = ridge_metrics_per_transect["pre_mig_dist"]

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 9))

ax1 = spacing_per_transect.boxplot(ax=ax1, grid=False)
ax2 = widths_per_transect.boxplot(ax=ax2, grid=False)
ax3 = amps_per_transect.boxplot(ax=ax3, grid=False)

# Set tick labels
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax3.set_xticklabels(spacing_per_transect.columns.str.replace("t_0*", "T", regex=True))

# Set y labels
ax1.set_ylabel("Ridge Spacing [m]", size=16)
ax2.set_ylabel("Ridge Widths [m]", size=16)
ax3.set_ylabel("Ridge Amps [m]", size=16)

# ax1.plot([0, 36], [120,120], color="r")

ax1.set_title("Ridge Metric Variation within Transects - Transect number increases downstream", size=20, horizontalalignment="center")
plt.tight_layout()
# plt.savefig(f"RidgeMetrics_Boxplot_Transect_{bend_id}.png", dpi=300)


In [ ]:
ridge_metrics.plot(x="ridge_amp", y="ridge_width", kind="scatter")

In [ ]:
ridge_metrics_per_ridge = ridge_metrics.unstack()

amps_per_ridge = ridge_metrics_per_ridge["ridge_amp"]
widths_per_ridge = ridge_metrics_per_ridge["ridge_width"]
spacing_per_ridge = ridge_metrics_per_ridge["pre_mig_dist"]

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 9))

ax1 = spacing_per_ridge.boxplot(ax=ax1, grid=False)
ax2 = widths_per_ridge.boxplot(ax=ax2, grid=False)
ax3 = amps_per_ridge.boxplot(ax=ax3, grid=False)

# Set tick labels
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax3.set_xticklabels(spacing_per_ridge.columns.str.replace("r_*", "R", regex=True))

# Set y labels
ax1.set_ylabel("Ridge Spacing [m]", size=16)
ax2.set_ylabel("Ridge Widths [m]", size=16)
ax3.set_ylabel("Ridge Amps [m]", size=16)

ax1.set_title("Ridge Metric Variation within Ridges - Ridge order is arbirary", size=20, horizontalalignment="center")
plt.tight_layout()
# plt.savefig(f"RidgeMetrics_Boxplot_Ridge_{bend_id}.png", dpi=300)

# Plot metrics aggregated (and normalized) to the ridge, transect and packet scales


In [ ]:
def normalize_vals(x):
    """Normalize the values within an array x between 0 and 1 """
    norm = (x - x.min())/(x.max()-x.min())
    return norm

In [ ]:
def agg_and_norm_itx_metrics(itx, ridges, transects, packets, rs_variables, agg_stat="median"):
    
    norm_variable_col_names = [f"{variable}_norm" for variable in rs_variables]

    # Normalize itx values
    itx[norm_variable_col_names] = itx[rs_variables].apply(normalize_vals)

    # Aggregate to the ridge scale
    ridge_agg = itx.groupby(level=-1).agg(agg_stat)
    ridge_agg[norm_variable_col_names] = ridge_agg[rs_variables].apply(normalize_vals)
    ridge_agg = ridge_agg.join(ridges.set_index("ridge_id").geometry)

    # Aggregate to the transect scale
    trans_agg = itx.groupby(level=0).agg(agg_stat)
    trans_agg[norm_variable_col_names] = trans_agg[rs_variables].apply(normalize_vals)
    trans_agg = trans_agg.join(transects.set_index("transect_id").geometry)

    # Normalize within the packets
    # Spatial join packet id onto itx points
    itx_packets = itx.sjoin(packets.drop("bend_id", axis=1))
    itx_packets = itx_packets.reset_index().set_index(["packet_id", "transect_id", "ridge_id"])

    # Normalize amplitude and width within packet bounds
    packet_norm = pd.DataFrame(columns=norm_variable_col_names)
    packet_norm[norm_variable_col_names] = itx_packets[rs_variables].groupby(level=0).apply(normalize_vals)

    # Join back the geometry anc recast to GDF
    packet_norm = gpd.GeoDataFrame(packet_norm.join(itx_packets.geometry, how="left"))

    return itx, ridge_agg, trans_agg, packet_norm

In [ ]:
itx, ridge_agg, trans_agg, packet_norm = agg_and_norm_itx_metrics(itx, ridges, transects, packets, ["ridge_amp", "ridge_width", "pre_mig_dist"])

In [ ]:
def adjust_ax_lim(min, max, perc):
    dist = max - min
    buff = dist*perc
    return(min-buff, max+buff)

rs_variable = "ridge_amp_norm"

# Make plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(6.5, 2))

# text_coords = (839355, 4311246)
text_coords = (839000, 4311350)

# Ridge plot
ridge_agg.plot(ax=ax1, column=rs_variable)  #, linewidth=ridge_agg["ridge_width"]/12)
ridges.plot(ax=ax1, color="k", ls="--", lw=0.5, zorder=0)

ax1.text(*text_coords, "Ridge-Scale\nAggregation", size=10)
# ax1.set_title("Ridge-Scale Aggregation")
ax1.set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
ax1.set_axis_off()

# Transect Plot
trans_agg.plot(ax=ax2, column=rs_variable, linewidth=1.5)
ridges.plot(ax=ax2, color="k", ls="--", lw=0.5, zorder=0)

ax2.text(*text_coords, "Transect-Scale\nAggregation", size=10)
# ax2.set_title("Transect-Scale Aggregation")
ax2.set_axis_off()
ax2.set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))

# Packet Plot
divider = make_axes_locatable(ax3)
cax = divider.append_axes("right", size="5%", pad=0.1, aspect=10)

# packet_norm.plot(ax=ax3, column="ridge_amp", markersize=packet_norm["ridge_width"]*8, legend=True, cax=cax)
packet_norm.plot(ax=ax3, column=rs_variable, markersize=3, legend=True, cax=cax)
ridges.plot(ax=ax3, color="k", ls="--", lw=0.5, zorder=0)
packets.boundary.plot(ax=ax3, color="k", lw=1, zorder=0)

min_max = [0,1]
cax.set_yticks(min_max, labels = np.round(min_max, 1), size=8)
cax.set_ylabel("Norm Amp", rotation=270, size=8)

ax3.text(*text_coords, "Packet-Scale\nNormalization", size=10)
# ax3.set_title("Packet-Scale Normalization")
ax3.set_axis_off()
ax3.set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))

# plt.suptitle("Aggregated Ridge Metrics", size=20)
plt.tight_layout()


# plt.savefig(f"MultiScaleMetrics_{bend_id}.png", dpi=300)

## Original plot

In [ ]:
# def adjust_ax_lim(min, max, perc):
#     dist = max - min
#     buff = dist*perc
#     return(min-buff, max+buff)

# # Make plot
# fig, axs = plt.subplots(2, 2, figsize=(6.5, 4.5))

# # text_coords = (839355, 4311246)
# text_coords = (839100, 4311350)



# # Regular ITX plot
# divider = make_axes_locatable(axs[0, 0])
# cax = divider.append_axes("right", size="5%", pad=0.1, aspect=4)

# vmin = 0
# vmax = itx["ridge_amp"].max()

# itx.plot(ax=axs[0, 0], column="ridge_amp", markersize=7, legend=True, cax=cax, vmin=vmin)
# ridges.plot(ax=axs[0, 0], color="k", ls="--", lw=0.5, zorder=0)
# # packets.boundary.plot(ax=axs[1,0], color="k", lw=1, zorder=0)

# min_max = [vmin, vmax]
# cax.set_yticks(min_max, labels = np.round(min_max, 1), size=8)
# cax.set_ylabel("Ridge Amplitude [m]", rotation=270, size=8)

# axs[0, 0].text(*text_coords, "Intersection-Scale\nMeasurements", size=10)
# axs[0, 0].set_axis_off()

# # Packet Plot
# divider = make_axes_locatable(axs[0, 1])
# cax = divider.append_axes("right", size="5%", pad=0.1, aspect=10)

# # packet_norm.plot(ax=axs[1,0], column="ridge_amp", markersize=packet_norm["ridge_width"]*8, legend=True, cax=cax)
# packet_norm.plot(ax=axs[0, 1], column="ridge_amp", markersize=7, legend=True, cax=cax)
# ridges.plot(ax=axs[0, 1], color="k", ls="--", lw=0.5, zorder=0)
# packets.boundary.plot(ax=axs[0, 1], color="k", lw=1, zorder=0)

# min_max = [0,1]
# cax.set_yticks(min_max, labels = np.round(min_max, 1), size=8)
# cax.set_ylabel("Normalized Amplitude", rotation=270, size=8)

# axs[0, 1].text(*text_coords, "Packet-Scale\nNormalization", size=10)
# # axs[1,0].set_title("Packet-Scale Normalization")
# axs[0, 1].set_axis_off()
# # axs[0, 1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))


# # Ridge plot
# ridge_agg.plot(ax=axs[1,0], column="ridge_amp_norm", linewidth=ridge_agg["ridge_width"]/12)
# ridges.plot(ax=axs[1,0], color="k", ls="--", lw=0.5, zorder=0)

# axs[1,0].text(*text_coords, "Ridge-Scale\nAggregation", size=10)
# # axs[0, 0].set_title("Ridge-Scale Aggregation")
# axs[1,0].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
# axs[1,0].set_axis_off()

# # Transect Plot
# trans_agg.plot(ax=axs[1,1], column="ridge_amp_norm", linewidth=1.5)
# ridges.plot(ax=axs[1,1], color="k", ls="--", lw=0.5, zorder=0)

# axs[1,1].text(*text_coords, "Transect-Scale\nAggregation", size=10)
# # axs[0, 1].set_title("Transect-Scale Aggregation")
# axs[1,1].set_axis_off()
# axs[1,1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))



# # plt.suptitle("Aggregated Ridge Metrics", size=20)
# plt.tight_layout()


# plt.savefig(f"MultiScaleMetrics_{bend_id}_2by2.png", dpi=300)

In [ ]:
packet_norm.columns

## Plot each variable in a 2 by 2 space

In [ ]:
def adjust_ax_lim(min, max, perc):
    dist = max - min
    buff = dist*perc
    return(min-buff, max+buff)

### Amplitude

In [ ]:
# =========== FIGURE SETUP ===========
rs_variable = "ridge_amp_norm"
rs_variable_title = "Ridge Amplitude"

# Make plot
fig, axs = plt.subplots(2, 2, figsize=(6.5, 4.5))

# Set subtitle loc
text_coords = (839100, 4311350)
label_coords = (839100, 4311650)
# ====================================



# Regular ITX plot
itx.plot(ax=axs[0, 0], column=rs_variable, markersize=7)
ridges.plot(ax=axs[0, 0], color="k", ls="--", lw=0.5, zorder=0)

axs[0,0].text(*text_coords, "Intersection-Scale\nMeasurements", size=10)
axs[0,0].set_axis_off()
axs[0,0].text(*label_coords, "a)", size=10)


# Packet Plot
# divider = make_axes_locatable(axs[0, 1])
# cax = divider.append_axes("right", size="5%", pad=0.1, aspect=10)

packet_norm.plot(ax=axs[1,0], column=rs_variable, markersize=7) #, legend=True, cax=cax)
ridges.plot(ax=axs[1,0], color="k", ls="--", lw=0.5, zorder=0)
packets.boundary.plot(ax=axs[1,0], color="k", lw=1, zorder=0)

# min_max = [1,0]
# cax.set_yticks(min_max, labels = np.round(min_max, 1), size=8)
# cax.set_ylabel("Normalized Amplitude", rotation=270, size=8)

axs[1,0].text(*text_coords, "Packet-Scale\nNormalization", size=10)
axs[1,0].set_axis_off()
axs[1,0].text(*label_coords, "c)", size=10)


# Ridge plot
ridge_agg.plot(ax=axs[0,1], column=rs_variable)
ridges.plot(ax=axs[0,1], color="k", ls="--", lw=0.5, zorder=0)

axs[0,1].text(*text_coords, "Ridge-Scale\nAggregation", size=10)
axs[0,1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
axs[0,1].set_axis_off()
axs[0,1].text(*label_coords, "b)", size=10)


# Transect Plot
trans_agg.plot(ax=axs[1,1], column=rs_variable, linewidth=1.5)
ridges.plot(ax=axs[1,1], color="k", ls="--", lw=0.5, zorder=0)

axs[1,1].text(*text_coords, "Transect-Scale\nAggregation", size=10)
axs[1,1].set_axis_off()
axs[1,1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
axs[1,1].text(*label_coords, "d)", size=10)


# Colorbar
cax = plt.axes((0.93, 0.5, 0.03, 0.15))
cbar = fig.colorbar(cm.ScalarMappable(cmap="viridis"), cax=cax)
cbar.set_ticks([0, 1])

plt.suptitle(f"Normalized {rs_variable_title}", size=16)
plt.tight_layout()


# plt.savefig(f"figures/MultiScaleMetrics_{bend_id}_{rs_variable}_2by2.png", dpi=300)

### Width

In [ ]:
# =========== FIGURE SETUP ===========
rs_variable = "ridge_amp_norm"
rs_variable_title = "Ridge Amplitude"

# Make plot
fig, axs = plt.subplots(2, 2, figsize=(6.5, 4.5))

# Set subtitle loc
text_coords = (839100, 4311350)
label_coords = (839100, 4311650)
# ====================================



# Regular ITX plot
itx.plot(ax=axs[0, 0], column=rs_variable, markersize=7)
ridges.plot(ax=axs[0, 0], color="k", ls="--", lw=0.5, zorder=0)

axs[0,0].text(*text_coords, "Intersection-Scale\nMeasurements", size=10)
axs[0,0].set_axis_off()
axs[0,0].text(*label_coords, "a)", size=10)


# Packet Plot
# divider = make_axes_locatable(axs[0, 1])
# cax = divider.append_axes("right", size="5%", pad=0.1, aspect=10)

packet_norm.plot(ax=axs[1,0], column=rs_variable, markersize=7) #, legend=True, cax=cax)
ridges.plot(ax=axs[1,0], color="k", ls="--", lw=0.5, zorder=0)
packets.boundary.plot(ax=axs[1,0], color="k", lw=1, zorder=0)

# min_max = [1,0]
# cax.set_yticks(min_max, labels = np.round(min_max, 1), size=8)
# cax.set_ylabel("Normalized Amplitude", rotation=270, size=8)

axs[1,0].text(*text_coords, "Packet-Scale\nNormalization", size=10)
axs[1,0].set_axis_off()
axs[1,0].text(*label_coords, "c)", size=10)


# Ridge plot
ridge_agg.plot(ax=axs[0,1], column=rs_variable)
ridges.plot(ax=axs[0,1], color="k", ls="--", lw=0.5, zorder=0)

axs[0,1].text(*text_coords, "Ridge-Scale\nAggregation", size=10)
axs[0,1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
axs[0,1].set_axis_off()
axs[0,1].text(*label_coords, "b)", size=10)


# Transect Plot
trans_agg.plot(ax=axs[1,1], column=rs_variable, linewidth=1.5)
ridges.plot(ax=axs[1,1], color="k", ls="--", lw=0.5, zorder=0)

axs[1,1].text(*text_coords, "Transect-Scale\nAggregation", size=10)
axs[1,1].set_axis_off()
axs[1,1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
axs[1,1].text(*label_coords, "d)", size=10)


# Colorbar
cax = plt.axes((0.93, 0.5, 0.03, 0.15))
cbar = fig.colorbar(cm.ScalarMappable(cmap="viridis"), cax=cax)
cbar.set_ticks([0, 1])

plt.suptitle(f"Normalized {rs_variable_title}", size=16)
plt.tight_layout()


# plt.savefig(f"figures/MultiScaleMetrics_{bend_id}_{rs_variable}_2by2.png", dpi=300)

### Migration Distance

In [ ]:
# =========== FIGURE SETUP ===========
rs_variable = "ridge_amp_norm"
rs_variable_title = "Ridge Amplitude"

# Make plot
fig, axs = plt.subplots(2, 2, figsize=(6.5, 4.5))

# Set subtitle loc
text_coords = (839100, 4311350)
label_coords = (839100, 4311650)
# ====================================



# Regular ITX plot
itx.plot(ax=axs[0, 0], column=rs_variable, markersize=7)
ridges.plot(ax=axs[0, 0], color="k", ls="--", lw=0.5, zorder=0)

axs[0,0].text(*text_coords, "Intersection-Scale\nMeasurements", size=10)
axs[0,0].set_axis_off()
axs[0,0].text(*label_coords, "a)", size=10)


# Packet Plot
# divider = make_axes_locatable(axs[0, 1])
# cax = divider.append_axes("right", size="5%", pad=0.1, aspect=10)

packet_norm.plot(ax=axs[1,0], column=rs_variable, markersize=7) #, legend=True, cax=cax)
ridges.plot(ax=axs[1,0], color="k", ls="--", lw=0.5, zorder=0)
packets.boundary.plot(ax=axs[1,0], color="k", lw=1, zorder=0)

# min_max = [1,0]
# cax.set_yticks(min_max, labels = np.round(min_max, 1), size=8)
# cax.set_ylabel("Normalized Amplitude", rotation=270, size=8)

axs[1,0].text(*text_coords, "Packet-Scale\nNormalization", size=10)
axs[1,0].set_axis_off()
axs[1,0].text(*label_coords, "c)", size=10)


# Ridge plot
ridge_agg.plot(ax=axs[0,1], column=rs_variable)
ridges.plot(ax=axs[0,1], color="k", ls="--", lw=0.5, zorder=0)

axs[0,1].text(*text_coords, "Ridge-Scale\nAggregation", size=10)
axs[0,1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
axs[0,1].set_axis_off()
axs[0,1].text(*label_coords, "b)", size=10)


# Transect Plot
trans_agg.plot(ax=axs[1,1], column=rs_variable, linewidth=1.5)
ridges.plot(ax=axs[1,1], color="k", ls="--", lw=0.5, zorder=0)

axs[1,1].text(*text_coords, "Transect-Scale\nAggregation", size=10)
axs[1,1].set_axis_off()
axs[1,1].set_ylim(*adjust_ax_lim(*ridges.total_bounds[1::2], 0.05))
axs[1,1].text(*label_coords, "d)", size=10)


# Colorbar
cax = plt.axes((0.93, 0.5, 0.03, 0.15))
cbar = fig.colorbar(cm.ScalarMappable(cmap="viridis"), cax=cax)
cbar.set_ticks([0, 1])

plt.suptitle(f"Normalized {rs_variable_title}", size=16)
plt.tight_layout()


# plt.savefig(f"figures/MultiScaleMetrics_{bend_id}_{rs_variable}_2by2.png", dpi=300)

In [ ]:
cm.ScalarMappable(cmap="viridis")

In [ ]:
ax = ridges.plot()
itx.xs("r_022", level=1).plot(ax=ax)

In [ ]:
itx.xs("r_022", level=1).pre_mig_dist.median()

In [ ]:
dist_itx = itx[[col for col in itx.columns if "mig_dist" in col]+["geometry"]]
dist_itx

In [ ]:
dist_itx.to_file(filename="itx_distances.geojson", index=True)

In [ ]:
trans_amp_range = trans_agg["ridge_amp"].max() - trans_agg["ridge_amp"].min()
ridge_amp_range = ridge_agg["ridge_amp"].max() - ridge_agg["ridge_amp"].min()

trans_width_range = trans_agg["ridge_width"].max() - trans_agg["ridge_width"].min()
ridge_width_range = ridge_agg["ridge_width"].max() - ridge_agg["ridge_width"].min()

print(f"Range for transect median amplitude: {trans_amp_range}")
print(f"Range for ridge median amplitude: {ridge_amp_range}")
print("\n")
print(f"Range for transect median width: {trans_width_range}")
print(f"Range for ridge median amplitude: {ridge_width_range}")

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6.5, 4))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1, aspect=5)

vmin = 0
vmax = itx["ridge_amp"].max()

itx.plot(ax=ax, column="ridge_amp", legend=True, cax=cax, vmin=vmin)
ridges.plot(ax=ax, color="k", ls="--", lw=0.5, zorder=0)
# packets.boundary.plot(ax=ax3, color="k", lw=1, zorder=0)

min_max = [vmin, vmax]
cax.set_yticks(min_max, labels = np.round(min_max, 1))
cax.set_ylabel("Ridge Amplitude [m]", rotation=270)

# ax.set_axis_off()


In [ ]:
def calc_iqr(x):
    q25, q75 = np.quantile(x, [0.25, 0.75])
    iqr = q75 - q25
    return iqr

In [ ]:
ridge_iqr = ridge_metrics["ridge_amp"].groupby(level=-1).apply(calc_iqr)
trans_iqr = ridge_metrics["ridge_amp"].groupby(level=0).apply(calc_iqr)

fig, ax = plt.subplots(1, 1)

ax.hist(ridge_iqr.values)
ax.hist(trans_iqr.values, alpha=0.5)